In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
import pickle
import time

In [ ]:
start_epoch = 1

In [ ]:
class LSTM_Model(nn.Module):
    def __init__(self, hidden_size, embedding_dim,vocab_size):
        super(LSTM_Model, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, 2)
        self.predictor = nn.Linear(hidden_size,1)
        
    def forward(self,data):
        
        emb_out = self.embedding(data)
        output, _ = self.lstm(emb_out)
        pred = self.predictor(output[:,-1,:])    ## Use the output from the last LSTM Cell
        label = torch.sigmoid(pred)
        
        return label


In [ ]:
model = torch.load("/content/drive/My Drive/models/SA_LSTM/model/e_{}.bin".format(str(start_epoch)))

In [ ]:
with open("/content/drive/My Drive/models/SA_LSTM/data_X.bin", "rb") as f:
    data_X = pickle.load(f)

with open("/content/drive/My Drive/models/SA_LSTM/data_Y.bin", "rb") as f:
    data_Y = pickle.load(f)


In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(data_X, data_Y, test_size=0.2)

In [ ]:
with open("/content/drive/My Drive/models/SA_LSTM/vocab.bin", "rb") as f:
    vocab = pickle.load(f)

In [ ]:
for i in range(len(train_X)):
    for j in range(len(train_X[i])):
        word = train_X[i][j]
        train_X[i][j] = vocab[word]

In [ ]:
for i in range(len(test_X)):
    for j in range(len(test_X[i])):
        word = test_X[i][j]
        test_X[i][j] = vocab[word]

In [ ]:
# def pad_trunc(arr, max_len):
#     for i in range(len(arr)):
#         temp = []
#         for j in range(max_len):
#             if j < len(arr[i]):
#                 temp.append(arr[i][j])
#             else:
#                 temp.append(0)
#         arr[i] = temp
#     return arr
def to_list_of_tensor(arr):
    ret_arr = []
    for tens in arr:
        tens = [tens]
        ret_arr.append(torch.tensor(tens))
    return ret_arr

In [ ]:
train_X = to_list_of_tensor(train_X)
train_Y = torch.tensor(train_Y)
test_X = to_list_of_tensor(test_X)
test_Y = torch.tensor(test_Y)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu") 

In [ ]:
def train(model, optimizer, criterion, start_epoch, train_X, train_Y, test_X, test_Y):
    start = time.time()
    for e in range(start_epoch,  start_epoch + 50):
        train_loss = 0
        train_acc = 0

        test_loss = 0
        test_acc = 0
        for i in range(len(train_X)):
            x = train_X[i].to(device)
            y = train_Y[i:i+1].to(device)
            y = y.float()
            optimizer.zero_grad()
            prediction = model(x)
            prediction = prediction.squeeze(1)
            loss = criterion(prediction,y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.data.item()
            train_acc += torch.eq(prediction.round(),y).sum().item()
        train_loss /= len(train_X)
        train_acc /= len(train_X)
        

        for i in range(len(test_X)):
            x = test_X[i].to(device)
            y = test_Y[i:i+1].to(device)
            y = y.float()
            prediction = model(x)
            prediction = prediction.squeeze(1)
            loss = criterion(prediction,y)

            test_loss += loss.data.item()
            test_acc += torch.eq(prediction.round(),y).sum().item()
        test_loss /= len(test_X)
        test_acc /= len(test_X)

        if (e+1)%10 == 0:
            torch.save(model, "/content/drive/My Drive/models/SA_LSTM/model/e_" + str(e+1)+".bin")
        duration = (time.time() - start ) / 60
        print("Epoch: {}  Training (loss,acc): ({},{})  Test (loss, acc):({},{})  Time: {}".format( e+1, train_loss, train_acc, test_loss, test_acc, duration ))


In [ ]:
vocab_size = len(vocab)
model = LSTM_Model(100,300, vocab_size)

model.to(device)

optimizer = torch.optim.SGD(model.parameters(), 0.01)
criterion = nn.BCELoss()

train(model, optimizer, criterion, start_epoch, train_X, train_Y, test_X, test_Y)

Epoch: 2  Training (loss,acc): (0.6864687737550587,0.53455)  Test (loss, acc):(0.669354436661303,0.5744)  Time: 5.265171360969544
Epoch: 3  Training (loss,acc): (0.6529640570515767,0.597475)  Test (loss, acc):(0.6546723191812635,0.5925)  Time: 10.528157969315847
Epoch: 4  Training (loss,acc): (0.6318401426361409,0.62735)  Test (loss, acc):(0.6519695862270891,0.5963)  Time: 15.799138553937276
Epoch: 5  Training (loss,acc): (0.618489934108546,0.6448)  Test (loss, acc):(0.6524149346228689,0.6017)  Time: 21.050539227326713
Epoch: 6  Training (loss,acc): (0.6086469382583164,0.65605)  Test (loss, acc):(0.6540057584639639,0.6031)  Time: 26.28072670698166
Epoch: 7  Training (loss,acc): (0.600954131556023,0.663975)  Test (loss, acc):(0.655915528306365,0.6005)  Time: 31.515857275327047
Epoch: 8  Training (loss,acc): (0.5946370130056282,0.668775)  Test (loss, acc):(0.6576764418825507,0.601)  Time: 36.74725758632024
Epoch: 9  Training (loss,acc): (0.5893494843515334,0.671025)  Test (loss, acc):(0.

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type LSTM_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 10  Training (loss,acc): (0.5848609350675251,0.67265)  Test (loss, acc):(0.6611891578637064,0.6055)  Time: 47.23220731417338
Epoch: 11  Training (loss,acc): (0.5809931913838023,0.6736)  Test (loss, acc):(0.6630538691155613,0.6066)  Time: 52.460407483577725
Epoch: 12  Training (loss,acc): (0.5776031043285038,0.674575)  Test (loss, acc):(0.6650233895076438,0.603)  Time: 57.678933227062224
Epoch: 13  Training (loss,acc): (0.5745555488599348,0.67495)  Test (loss, acc):(0.6672911601988599,0.6025)  Time: 62.906925829251605
Epoch: 14  Training (loss,acc): (0.5718408000110067,0.675375)  Test (loss, acc):(0.6697330732550472,0.6018)  Time: 68.1239775578181
Epoch: 15  Training (loss,acc): (0.5693374046549783,0.67575)  Test (loss, acc):(0.672243147637602,0.6017)  Time: 73.35304605960846
Epoch: 16  Training (loss,acc): (0.5670284673932474,0.67575)  Test (loss, acc):(0.6747465233644471,0.6017)  Time: 78.59035943746566
Epoch: 17  Training (loss,acc): (0.5648865740240144,0.676575)  Test (loss, 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)